# Prétraitement des images

Le notebook va chercher à trouver des prétraitements d'images visant à améliorer les performances en classification par création de features par algorithme SIFT, et clustering par la méthode des k-means.

## Dépendances

In [9]:
import numpy as np
import pandas as pd
import cv2

df = pd.read_csv('base_de_données.csv')
photos = df['photo']

photo_givrage = photos[43] #108 originalement
photo_tache = photo_givrage
photo_coulure = photos[113]

sift = cv2.SIFT_create()


In [11]:
image = cv2.imread('./Photographies pieces chaudes/' + photo_tache ,cv2.IMREAD_GRAYSCALE)
print(image.dtype)
# cv2.imshow('img',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

uint8


In [3]:
#Rognage de l'image
img_rogne = image[350:900,600:1200]
print(img_rogne.shape)
cv2.imshow('img',img_rogne,)
cv2.waitKey(0)
cv2.destroyAllWindows()

(550, 600)


In [ ]:
#Rognage extreme
xtreme = image[800:900,630:900]
cv2.imshow('img',xtreme)
cv2.waitKey(0)
cv2.destroyAllWindows()

kp = sift.detect(xtreme)
img_=cv2.drawKeypoints(xtreme,kp,xtreme)
cv2.imwrite('sift_keypoints_xtreme.jpg',img_)

In [4]:
#SIFT en utilisant un masque

mask = np.zeros_like(image)
z = np.ones((550,600))
mask[350:900,600:1200] = z
mask = np.uint8(mask)
print(mask.shape)

kp = sift.detect(image,mask = mask)
img_=cv2.drawKeypoints(image,kp,image)
cv2.imwrite('sift_keypoints_mask.jpg',img_)

kp = sift.detect(image)
img_=cv2.drawKeypoints(image,kp,image)
cv2.imwrite('sift_keypoints.jpg',img_)


(1280, 1920)


True

On remarque que même avec un masque, SIFT ne mets pas de keypoints là où sont les défauts.  
On va essayer différentes méthodes pour souligner les défauts.

In [ ]:
image = cv2.imread('./Photographies pieces chaudes/' + photo_coulure ,cv2.IMREAD_GRAYSCALE)

#Augmentation du contraste
alpha = 1
img_rogne = image[250:1050,530:1300]
img_rogne = np.clip((np.round(alpha*img_rogne,decimals=0)),0,255)
img_rogne = img_rogne.astype(np.uint8)


print(img_rogne.shape)
cv2.imshow('img',img_rogne)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#Methode du gamma

img_rogne = image[250:1050,530:1300]

gamma = 1.5
lookUpTable = np.empty((1,256), np.uint8)
for i in range(256):
	lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)

res = cv2.LUT(img_rogne, lookUpTable)
cv2.imshow('img',res)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#Test des key points sur l'image avec gamma modifié

kp = sift.detect(res)
img_=cv2.drawKeypoints(res,kp,res)
cv2.imwrite('sift_keypoints_gamma.jpg',img_)


Changer le gamma ne donne pas non plus de résultats concluants.

Changer le contraste semble pas terrible. Essayons de trouver les contours avec la méthode de Canny.

In [ ]:
contours = cv2.Canny(img_rogne,10,100)
cv2.imshow('img',contours)
cv2.waitKey(0)
cv2.destroyAllWindows()

Pas fameux non plus.

On va essayer de modifier les paramètres de l'algorithme SIFT pour garder des features qui nous interessent.

In [12]:
sift_mod = cv2.SIFT_create(edgeThreshold=2,contrastThreshold=0.014,sigma=0.5)

kp = sift_mod.detect(image,mask = mask)
img_=cv2.drawKeypoints(image,kp,image)
cv2.imwrite('sift_keypoints_edgeth.jpg',img_)

#semble etre un bon réglage pour détecter les tâches

True

Essayons de mélanger la technique du gamma avec les réglages des paramètres de SIFT

In [ ]:
sift_mod = cv2.SIFT_create(edgeThreshold=2,contrastThreshold=0.012,sigma=1)

kp = sift_mod.detect(res)
img_=cv2.drawKeypoints(res,kp,res)
cv2.imwrite('sift_mod_gamma.jpg',img_)